# Проверяем можно ли пересобрать модельку в onnx с 400мс буфером

In [13]:
from tone import StreamingCTCPipeline, read_stream_example_audio


pipeline = StreamingCTCPipeline.from_hugging_face()

# state = None  # Current state of the ASR pipeline (None - initial)
# for audio_chunk in read_stream_example_audio():  # Use any source of audio chunks
#     new_phrases, state = pipeline.forward(audio_chunk, state)
#     # print(new_phrases)

# # Finalize the pipeline and get the remaining phrases
# new_phrases, _ = pipeline.finalize(state)
# print(new_phrases)

In [14]:
# Хорошо, давай попробуем теперь скачать модельку локально и по-трейсить
from pathlib import Path
resources_path = Path("./model")
resources_path.mkdir(parents=True, exist_ok=True) 

# StreamingCTCPipeline.download_from_hugging_face(dir_path="./model")

In [15]:
# ! python3 -m tone.scripts.export --chunk-duration-ms 300 --output_path ./model_300ms.onnx

In [16]:
import tone.scripts.export as tone_onnx_export

path_to_pretrained = "t-tech/T-one"
chunk_duration_ms = 300

# model = tone_onnx_export.ModelToExport(path_to_pretrained, chunk_duration_ms)
# model_bytes = tone_onnx_export._export_onnx(model)

# output_path = Path("./model_300ms.onnx")
# output_path.write_bytes(model_bytes)

In [17]:
chunk_duration_ms = 400

model = tone_onnx_export.ModelToExport(path_to_pretrained, chunk_duration_ms)
model_bytes = tone_onnx_export._export_onnx(model)

output_path = Path(f"./model_{chunk_duration_ms}ms.onnx")
output_path.write_bytes(model_bytes)

signal.shape=torch.Size([5, 3200, 1])
signal.shape=torch.Size([5, 3200, 1])


144198453

# Проверяем метрики в стриминге

как там wer считать ...

In [3]:
import json
from pathlib import Path

from tone import read_stream_audio, StreamingCTCPipeline, StreamingCTCModel
from nemo.collections.asr.metrics.wer import word_error_rate
from tqdm.auto import tqdm


# StreamingCTCPipeline.download_from_hugging_face("./model")
# pipeline = StreamingCTCPipeline.from_local("./model", providers=['CUDAExecutionProvider'])

StreamingCTCModel.AUDIO_CHUNK_SAMPLES = int(0.4 * StreamingCTCModel.SAMPLE_RATE)
StreamingCTCPipeline.CHUNK_SIZE = StreamingCTCModel.AUDIO_CHUNK_SAMPLES
pipeline = StreamingCTCPipeline.from_local("./model_400ms_fixed/streaming_acoustic/1", providers=['CUDAExecutionProvider'])

2025-12-22 13:01:08.492047315 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 38 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-12-22 13:01:08.500086461 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-12-22 13:01:08.500094601 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [4]:
SUB = str.maketrans('ё', 'е')
def clean_text(s: str) -> str:
    """Lowercase a string and substitute."""
    return s.lower().translate(SUB)
base_data_path = Path("/home/mle/aiphoria-asr-training/data/rus_finetune")

dataset_name = "test_collection"
# load collections manifest
with open(base_data_path / dataset_name / "tarred_audio_manifest.json") as fin:
    jdata = []
    for line in fin:
        jline = json.loads(line.strip())
        jdata.append(jline)
len(jdata)

hypos = []
gts = []
for _meta in tqdm(jdata):
    audio_path = base_data_path / dataset_name / Path(_meta["audio_filepath"]).name
    
    state = None
    chunk_phrases = []
    for audio_chunk in read_stream_audio(path_to_file=audio_path, chunk_size=pipeline.CHUNK_SIZE):
        new_phrases, state = pipeline.forward(audio_chunk, state)
        # print(state)
        print(state[0].shape)
        print(state[1].past_logprobs.shape[0])
        if new_phrases:
            chunk_phrases += new_phrases

    break
    
    # Finalize the pipeline and get the remaining phrases
    new_phrases, _ = pipeline.finalize(state)
    output = chunk_phrases + new_phrases
    pred_text = " ".join([phrase.text for phrase in output])
    hypos.append(clean_text(pred_text))
    
    gts.append(clean_text(_meta["text"]))

all_hypos += hypos
all_gts += gts
word_error_rate(hypos, gts)

  0%|          | 0/264 [00:02<?, ?it/s]

(1, 219729)
3
(1, 219729)
3
(1, 219729)
16
(1, 219729)
29
(1, 219729)
42
(1, 219729)
3
(1, 219729)
16
(1, 219729)
29
(1, 219729)
42
(1, 219729)
55
(1, 219729)
68
(1, 219729)
81


NameError: name 'all_hypos' is not defined

# Теперь считаем нормальные тесты

In [5]:
all_hypos = []
all_gts = []

In [3]:
SUB = str.maketrans('ё', 'е')
def clean_text(s: str) -> str:
    """Lowercase a string and substitute."""
    return s.lower().translate(SUB)
base_data_path = Path("/home/mle/aiphoria-asr-training/data/rus_finetune")

dataset_name = "test_collection"
# load collections manifest
with open(base_data_path / dataset_name / "tarred_audio_manifest.json") as fin:
    jdata = []
    for line in fin:
        jline = json.loads(line.strip())
        jdata.append(jline)
len(jdata)

hypos = []
gts = []
for _meta in tqdm(jdata):
    audio_path = base_data_path / dataset_name / Path(_meta["audio_filepath"]).name
    
    state = None
    chunk_phrases = []
    for audio_chunk in read_stream_audio(path_to_file=audio_path, chunk_size=pipeline.CHUNK_SIZE):
        new_phrases, state = pipeline.forward(audio_chunk, state)
        if new_phrases:
            chunk_phrases += new_phrases
    # Finalize the pipeline and get the remaining phrases
    new_phrases, _ = pipeline.finalize(state)
    output = chunk_phrases + new_phrases
    pred_text = " ".join([phrase.text for phrase in output])
    hypos.append(clean_text(pred_text))
    
    gts.append(clean_text(_meta["text"]))

all_hypos += hypos
all_gts += gts
word_error_rate(hypos, gts)

100%|██████████| 264/264 [00:33<00:00,  7.96it/s]


0.15128504672897197

In [6]:
dataset_name = "test_rec_support"
# load collections manifest
with open(base_data_path / dataset_name / "tarred_audio_manifest.json") as fin:
    jdata = []
    for line in fin:
        jline = json.loads(line.strip())
        jdata.append(jline)
len(jdata)

hypos = []
gts = []
for _meta in tqdm(jdata):
    audio_path = base_data_path / dataset_name / Path(_meta["audio_filepath"]).name
    
    state = None
    chunk_phrases = []
    for audio_chunk in read_stream_audio(path_to_file=audio_path, chunk_size=pipeline.CHUNK_SIZE):
        new_phrases, state = pipeline.forward(audio_chunk, state)
        if new_phrases:
            chunk_phrases += new_phrases
    # Finalize the pipeline and get the remaining phrases
    new_phrases, _ = pipeline.finalize(state)
    output = chunk_phrases + new_phrases
    pred_text = " ".join([phrase.text for phrase in output])
    hypos.append(clean_text(pred_text))
    
    gts.append(clean_text(_meta["text"]))

all_hypos += hypos
all_gts += gts
word_error_rate(hypos, gts)

100%|██████████| 311/311 [00:55<00:00,  5.62it/s]


0.12844488188976377

In [5]:
dataset_name = "test_support"
# load collections manifest
with open(base_data_path / dataset_name / "tarred_audio_manifest.json") as fin:
    jdata = []
    for line in fin:
        jline = json.loads(line.strip())
        jdata.append(jline)
len(jdata)

hypos = []
gts = []
for _meta in tqdm(jdata):
    audio_path = base_data_path / dataset_name / Path(_meta["audio_filepath"]).name
    
    state = None
    chunk_phrases = []
    for audio_chunk in read_stream_audio(path_to_file=audio_path, chunk_size=pipeline.CHUNK_SIZE):
        new_phrases, state = pipeline.forward(audio_chunk, state)
        if new_phrases:
            chunk_phrases += new_phrases
    # Finalize the pipeline and get the remaining phrases
    new_phrases, _ = pipeline.finalize(state)
    output = chunk_phrases + new_phrases
    pred_text = " ".join([phrase.text for phrase in output])
    hypos.append(clean_text(pred_text))
    
    gts.append(clean_text(_meta["text"]))

all_hypos += hypos
all_gts += gts
word_error_rate(hypos, gts)

100%|██████████| 314/314 [00:37<00:00,  8.36it/s]


0.19408054342552158

In [6]:
word_error_rate(all_hypos, all_gts)

0.15848406546080965